In [2]:
!pip install textblob

DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/jupyter-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 6.1 MB/s eta 0:00:00a 0:00:01


In [3]:
import pandas as pd
import regex as re
import glob
import zipfile
from textblob import TextBlob

In [4]:
# Read in your split of data
all_english_comments = pd.read_csv('Comments DB/English/FINAL/splits_initial/comments_processed_split_0.csv')
all_english_comments_original = pd.read_csv('Comments DB/English/FINAL/splits_initial/comments_original_split_0.csv')

In [22]:
# We saw after scraping that we still have some problematic comments, which we will now adress

In [5]:
# Remove comments with words like "video" and "channel" as they are associated with comments such as 'great video!'
# Note we also need to remove from original, because in this case we are removing whole comments !
all_english_comments = all_english_comments[~all_english_comments['Comment'].str.contains('video|channel', case=False)]
all_english_comments_original = all_english_comments_original[~all_english_comments_original['Comment'].str.contains('video|channel', case=False)]

In [6]:
# Remove numbers from comments
all_english_comments['Comment'] = all_english_comments['Comment'].apply(lambda x: re.sub(r'\d+', '', x))

In [7]:
# Remove words that have the scheme '@something' (i.e. remove words that begin with '@')
all_english_comments['Comment'] = all_english_comments['Comment'].apply(lambda x: re.sub(r'@\w+', '', x))


In [8]:
# Replace special characters (all but period ('.') and whitespaces) with nothing (so they are removed)
all_english_comments['Comment'] = all_english_comments['Comment'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s.]', '', x))
# Replace period with whitespace (so words are not getting merged)
all_english_comments['Comment'] = all_english_comments['Comment'].apply(lambda x: re.sub(r'\.', ' ', x))
# Remove trailing whitespaces
all_english_comments['Comment'] = all_english_comments['Comment'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())




In [9]:
# We also aim to fix spelling mistakes, as they can mess up our sentiment analysis heavily (we tested that on some models and got completely wrong sentiments due to spelling mistakes)
# This we can also do on the original comments directly 
# This will later on also be crucial for translation into other languages !

# Function to correct spelling mistakes
def correct_spelling(text):
    try:
        corrected_text = str(TextBlob(text).correct())
        return corrected_text
    except Exception as e:
        return text
    
all_english_comments['Comment'] = all_english_comments['Comment'].apply(correct_spelling)
all_english_comments_original['Comment'] = all_english_comments_original['Comment'].apply(correct_spelling)



In [10]:
# We first need to concatenate the two dataframes
# We will then drop duplicates
# We will then split them again

all_english_comments_combined = pd.concat([all_english_comments.reset_index(drop=True), all_english_comments_original.reset_index(drop=True)], axis = 1)
all_english_comments_combined.columns = ['Comment processed', 'Comment original']
all_english_comments_combined = all_english_comments_combined.drop_duplicates(subset='Comment processed', keep='first')
# Split the dataframes again
all_english_comments = all_english_comments_combined[['Comment processed']].rename(columns={'Comment processed': 'Comment'})
all_english_comments_original = all_english_comments_combined[['Comment original']].rename(columns={'Comment original': 'Comment'})

In [11]:
# Save to csv
all_english_comments.to_csv('Comments DB/English/FINAL/splits_processed/comments_processed_split_0.csv')
all_english_comments_original.to_csv('Comments DB/English/FINAL/splits_processed/comments_original_split_0.csv')